In [93]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

website = 'https://store.steampowered.com/search/?filter=topsellers'

response = requests.get(website)
if response.status_code == 200:
    page_source = response.content
else:
    print("Failed to fetch webpage:", response.status_code)
    exit()
    
soup = BeautifulSoup(page_source, features="html.parser")

driver = webdriver.Firefox()
driver.get(website)

wait = WebDriverWait(driver, 10)
wait.until(EC.url_to_be(website))

WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.ID, "acceptAllButton")))
driver.find_element(By.ID, 'acceptAllButton').click()

lambdaBsFind = lambda bsVariable, htmlType, className: bsVariable.find(htmlType, class_=className)

titleprices = soup.find_all('div', class_='responsive_search_nam`e_combined')
counter = 1
gdf = pd.DataFrame()
gdf['game'] = ''
gdf['price'] = 0
gdf['discout'] = ''
for titleprice in titleprices:
    title = lambdaBsFind(titleprice, 'span', 'title').getText()
    price = lambdaBsFind(titleprice, 'div', 'discount_final_price').getText()
    priceNum = re.sub(r'(Free)', '0', price)
    priceNum = re.sub(r'(zł)|\s*', '', priceNum)
    priceNum = re.sub(r'\,', '.', priceNum)
    priceNum = float(priceNum)
    discount = lambdaBsFind(titleprice, 'div', 'discount_pct')
    if discount is None:
        discount = '0%'
    else:
        discount = discount.getText()
    print('The ' + str(counter) + ' game ' + title + ' is for ' + price + ' Złoty with a discount of ' + discount)
    gdf.loc[len(gdf.index)] = [title, priceNum, discount]
    counter += 1

In [95]:
# The Pandas gameframe Top Sellers 
gdf

,game,price,discout
